In [1]:
### prepare input expression matrix for STIP
library(STIP)
library(Seurat)
library(ggplot2)
library(dplyr)
library(gridExtra)
library(clusterProfiler)







Attaching SeuratObject



### Load data and extract expression matrix

In [ ]:
# Load seurat object
obj = readRDS("path_to_seurat_object.rds")
# Get cell sequence according to pseudotime
cells_order = rownames(obj@meta.data[order(obj@meta.data$pseudotime), ]) ### The column "pseudotime" should be add to metadata before analysis
# Extract expression matrix
expr_df = obj@assays$RNA@data[, cells_order]

### Perform STIP gene fitting and plotting

In [ ]:
# Fit gene expression
fit_data_list <- STIP::fitData(as.matrix(expr_df), mc.cores = 4)
# Get gene expreesion pattern
gene_group <- STIP::genePattern(as.data.frame(fit_data_list[["data"]]))

# save result
stip_list <- list()
stip_list[["expr_df"]] <- expr_df
stip_list[["fit_data_list"]] <- fit_data_list
stip_list[["gene_group"]] <- gene_group

# Heatmap to view interested genes
### with q value cutoff as 0.05
fit_data <- stip_list[["fit_data_list"]][["data"]][stip_list[["fit_data_list"]][["qval"]] <= 0.05, ]
gene_group <- stip_list[["gene_group"]][rownames(fit_data), ]
gene_group <- gene_group[order(gene_group$pattern, gene_group$rank_point), ]

markers <- c("Pax7", "Vcam1", "Itga7", "Cd34", "Sdc4", "Myod1",
             "Des", "Mest", "Cdkn1c", "Igfbp5", "Myf5", "Myh1", 
             "Acta1", "Myl1", "Top2a", "Mki67", "Spry1", 
             "Myog", "Ckm", "Myh15", "Myh1") ### Gene list you are interested, please 
markers <- intersect(rownames(gene_group), markers)

### Plot and save the heatmap, should spend little while
output = "./"
pdf(paste0(output, "STIP_Heatmap.pdf"), width = 7, height=10)
p <- STIP::HeatmapSTIP(x=fit_data, gl=markers, annotation=as.matrix(gene_group)[, "pattern"])
ComplexHeatmap::draw(p)
dev.off()

### Gene ontology analysis

In [ ]:
### GO analysis of each group
### Group Enrichment analysis of STIP, qvalue 0.05

output <- "./"

universe <- rownames(expr_df)
fit_data <- stip_list[["fit_data_list"]][["data"]][stip_list[["fit_data_list"]][["qval"]] <= 0.05, ]
gene_group <- stip_list[["gene_group"]][rownames(fit_data), ]
gene_group <- gene_group[order(gene_group$pattern, gene_group$rank_point), ]

enrich_group_list <- list()
enrich_qval_list <- list()

# Enrichment for each group of genes
for (gp in unique(gene_group$pattern)){
    enrich_group <- STIP::enrichPattern(gene_group, gp, "mouse", universe=universe)
    enrich_group@result[, "EnrichRatio"] <- (as.numeric(sapply(strsplit(enrich_group@result$GeneRatio, "/"), function(i) i[1])) * as.numeric(sapply(strsplit(enrich_group@result$BgRatio, "/"), function(i) i[2]))) / (as.numeric(sapply(strsplit(enrich_group@result$GeneRatio, "/"), function(i) i[2])) * as.numeric(sapply(strsplit(enrich_group@result$BgRatio, "/"), function(i) i[1])))
    enrich_group_list[[gp]] <- enrich_group
    enrich_qval_list[[gp]] <- data.frame(enrich_group@result %>% filter(qvalue<=0.05) %>% arrange(qvalue))
}

# save enrichment result
saveRDS(enrich_group_list, paste0(output, "STIP_group_enrichment.rds"))
openxlsx::write.xlsx(x = enrich_qval_list, file = paste0(output, "STIP_group_enrichment.xlsx"), colNames = TRUE, rowNames = TRUE)

In [ ]:
### Bin Enrichment analysis of STIP, qvalue 0.05
### Get the number of genes in each group first, then specify bin.width
### Let each bin be 100-500 genes

enrich_bin_list <- list()
enrich_qval_list <- list()

pattern <- "D"
bin.width <- 0.1
stride <- 0.05
genes_bin_enrich <- STIP::compareEnrichBin(gene_group, pattern, bin.width=bin.width, stride=stride, species="mouse", ont="BP", universe=universe)
genes_bin_enrich@compareClusterResult[, "EnrichRatio"] <- (as.numeric(sapply(strsplit(genes_bin_enrich@compareClusterResult$GeneRatio, "/"), function(i) i[1])) * as.numeric(sapply(strsplit(genes_bin_enrich@compareClusterResult$BgRatio, "/"), function(i) i[2]))) / (as.numeric(sapply(strsplit(genes_bin_enrich@compareClusterResult$GeneRatio, "/"), function(i) i[2])) * as.numeric(sapply(strsplit(genes_bin_enrich@compareClusterResult$BgRatio, "/"), function(i) i[1])))
enrich_bin_list[[pattern]] <- genes_bin_enrich
enrich_qval_list[[pattern]] <- data.frame(genes_bin_enrich@compareClusterResult %>% filter(qvalue<=0.05) %>% arrange(Cluster, qvalue))

pattern <- "DI"
bin.width <- 0.2
stride <- 0.1
genes_bin_enrich <- STIP::compareEnrichBin(gene_group, pattern, bin.width=bin.width, stride=stride, species="mouse", ont="BP", universe=universe)
genes_bin_enrich@compareClusterResult[, "EnrichRatio"] <- (as.numeric(sapply(strsplit(genes_bin_enrich@compareClusterResult$GeneRatio, "/"), function(i) i[1])) * as.numeric(sapply(strsplit(genes_bin_enrich@compareClusterResult$BgRatio, "/"), function(i) i[2]))) / (as.numeric(sapply(strsplit(genes_bin_enrich@compareClusterResult$GeneRatio, "/"), function(i) i[2])) * as.numeric(sapply(strsplit(genes_bin_enrich@compareClusterResult$BgRatio, "/"), function(i) i[1])))
enrich_bin_list[[pattern]] <- genes_bin_enrich
enrich_qval_list[[pattern]] <- data.frame(genes_bin_enrich@compareClusterResult %>% filter(qvalue<=0.05) %>% arrange(Cluster, qvalue))

pattern <- "I"
bin.width <- 0.1
stride <- 0.05
genes_bin_enrich <- STIP::compareEnrichBin(gene_group, pattern, bin.width=bin.width, stride=stride, species="mouse", ont="BP", universe=universe)
genes_bin_enrich@compareClusterResult[, "EnrichRatio"] <- (as.numeric(sapply(strsplit(genes_bin_enrich@compareClusterResult$GeneRatio, "/"), function(i) i[1])) * as.numeric(sapply(strsplit(genes_bin_enrich@compareClusterResult$BgRatio, "/"), function(i) i[2]))) / (as.numeric(sapply(strsplit(genes_bin_enrich@compareClusterResult$GeneRatio, "/"), function(i) i[2])) * as.numeric(sapply(strsplit(genes_bin_enrich@compareClusterResult$BgRatio, "/"), function(i) i[1])))
enrich_bin_list[[pattern]] <- genes_bin_enrich
enrich_qval_list[[pattern]] <- data.frame(genes_bin_enrich@compareClusterResult %>% filter(qvalue<=0.05) %>% arrange(Cluster, qvalue))

pattern <- "ID"
bin.width <- 0.1
stride <- 0.05
genes_bin_enrich <- STIP::compareEnrichBin(gene_group, pattern, bin.width=bin.width, stride=stride, species="mouse", ont="BP", universe=universe)
genes_bin_enrich@compareClusterResult[, "EnrichRatio"] <- (as.numeric(sapply(strsplit(genes_bin_enrich@compareClusterResult$GeneRatio, "/"), function(i) i[1])) * as.numeric(sapply(strsplit(genes_bin_enrich@compareClusterResult$BgRatio, "/"), function(i) i[2]))) / (as.numeric(sapply(strsplit(genes_bin_enrich@compareClusterResult$GeneRatio, "/"), function(i) i[2])) * as.numeric(sapply(strsplit(genes_bin_enrich@compareClusterResult$BgRatio, "/"), function(i) i[1])))
enrich_bin_list[[pattern]] <- genes_bin_enrich
enrich_qval_list[[pattern]] <- data.frame(genes_bin_enrich@compareClusterResult %>% filter(qvalue<=0.05) %>% arrange(Cluster, qvalue))

saveRDS(enrich_bin_list, paste0(output, "STIP_bin_enrichment.rds"))
openxlsx::write.xlsx(x = enrich_qval_list, file = paste0(output, "STIP_bin_enrichment.xlsx"), colNames = TRUE, rowNames = TRUE)